## SD on EKS 部署

### 一. 基本环境初始化

#### 1. 安装基本命令

In [ ]:
%%bash
sudo chmod +x init-env.sh
./init-env.sh

#### 2. clone 代码

In [ ]:
%%bash
git clone --recurse-submodules https://Private_token@github.com/bnusunny/stable-diffusion-on-eks.git
cd ~/SageMaker/SDonEKS-Deploy/stable-diffusion-on-eks

#### 3. 创建模型存储桶,记录下桶的arn 后面会用到

In [13]:
BUCKET_NAME = "sd-models-XXX" #修改为需要创建的模型桶名称

In [ ]:
!aws s3 mb s3://$BUCKET_NAME --region us-west-2

记录下S3 ARN地址

In [14]:
! echo "arn:aws:s3::$BUCKET_NAME"

arn:aws:s3::sd-models-uw2


### 二. 镜像及Snapshot制备

#### 1. 镜像制作

In [ ]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
os.chdir('/home/ec2-user/SageMaker/SDonEKS-Deploy/tools')
cmd = "/bin/sh ./build_and_push.sh %s" %(region_name)
os.system(cmd)

#### 2. Snapshot 制作(这一步是可选项,建议在业务代码开发调试完成后再做)

In [ ]:
import boto3,os
account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name
Infer_API_IMG= "%s.dkr.ecr.%s.amazonaws.com/inference-api:latest" %(account_id,region_name)
Queue_Agent_IMG= "%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/queue-agent:latest" %(account_id,region_name)
OCI_URL = "oci://%s.dkr.ecr.%s.amazonaws.com/sd-on-eks/charts/" %(account_id,region_name)
print ("OCI_URL is :",OCI_URL)
os.chdir('/home/ec2-user/SageMaker/SDonEKS-Deploy/stable-diffusion-on-eks/utils/bottlerocket-images-cache')
cmd = "/bin/sh ./snapshot.sh %s %s" %(Infer_API_IMG,Queue_Agent_IMG)
os.system(cmd)

In [ ]:
#记录上面输出的snapshot id

### 三. CDK 部署

#### 修改 config.yaml文件,参考编辑Sample目录下的config.yaml文件,注意标点和格式

根据实际情况修改,S3桶arn,runtime 模型,自建ECR,Snapshot ID(如果启用)等.编辑后保存

以下代码需要在Terminal中执行

In [ ]:
%%bash
cd $HOME/SageMaker/SDonEKS-Deploy/
sudo chmod +x cdk_deploy.sh
./cdk_deploy.sh

执行完毕后从cloudformation输出记录 APIGateway 地址,形如https://c1544mqj.execute-api.<Your_Region>.amazonaws.com/prod/

执行完毕后从cloudformation输出记录 eks config update命令,如下,拷贝到bash 中执行:

```
aws eks update-kubeconfig --name SdOnEksDataPlaneStack --region <Your_Region> --role-arn arn:aws:iam::<Your_Account>:role/SdOnEksDataPlaneStack-SdOnEksDataPlaneStackAccessR-XXXXX
```

### 四. 模型下载同步

#### 1. 模型下载

找到tools目录下S3uploader文件夹,修改其中的down.csv 修改其中的url,bucket(models bucket)和目标key列后保存
执行

```
python3 s3uploader.py down.csv
```
开始下载

#### 2. 模型同步

进入datasync 访问控制台中找到历史记录是否有执行记录.

#### 3. 请求测试

模型同步完成后,即可使用Postman等工具对API Gateway请求测试.

#### 五.排错及管理

1. 首批pod可能由于模型加载错误导致无法启动可以使用如下命令重启pod

```
kubectl scale deployment sdruntime2-sd-on-eks-sd-webui-inference-api --replicas=0 -n sdruntime2
kubectl scale deployment sdruntime1-sd-on-eks-sd-webui-inference-api --replicas=0 -n sdruntime1
kubectl scale deployment sdruntime1-sd-on-eks-sd-webui-inference-api --replicas=1 -n sdruntime1
kubectl scale deployment sdruntime2-sd-on-eks-sd-webui-inference-api --replicas=1 -n sdruntime2
```

2. 如果Pod处于pending状态,请检查一下命令输出是否有异常

```
kc describe pod sdruntime1-sd-on-eks-sd-webui-inference-api-74f7cb554c-s6jh4 -n sdruntime1
kubectl logs -n karpenter -l app.kubernetes.io/name=karpenter
kubectl logs karpenter-76c95bd775-rdw6q  -n karpenter
```

3. 如果Pod启动出错,请检查一下命令输出是否有异常

```
kubectl logs sdruntime1-sd-on-eks-sd-webui-inference-api-74f7cb554c-t47rq -c sd-webui-queue-agent -n sdruntime1
kubectl exec -it  sdruntime1-sd-on-eks-sd-webui-inference-api-cb57f64fd-gt48p -c sd-webui-queue-agent -n sdruntime1 /bin/bash
kubectl logs sdruntime1-sd-on-eks-sd-webui-inference-api-cb57f64fd-gt48p -c sd-webui-queue-agent -n sdruntime2
```

4. 如果需要临时关闭keda的自动伸缩,停掉所有Wokernode,可以将keda deployments和 runtime deployment scale 到 0.